In [300]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import json
import pickle

In [301]:
df = pd.read_csv('data/test.csv')
df.shape

(470, 28)

In [302]:
df.columns

Index(['Id', 'Age', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EmployeeNumber',
       'EnvironmentSatisfaction', 'Gender', 'JobInvolvement', 'JobRole',
       'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome',
       'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager', 'CommunicationSkill',
       'Behaviour'],
      dtype='object')

### Feature Engineering

In [303]:
def extract_feature(df_input):
    df = df_input.copy()
    df['OwnStocks'] = df.StockOptionLevel.apply(lambda x: 'No' if x == 0 else 'Yes')
    df['PropWorkLife'] = df.TotalWorkingYears / df.Age
    df['PropExpComp'] = df.NumCompaniesWorked / (df.TotalWorkingYears+1)
    df['PropRoleComp'] = df.YearsInCurrentRole / (df.YearsAtCompany + 1)
    df['AgeBar'] = pd.cut(df.Age, bins = [0,27,45,np.inf], labels=['Young','Mid','Old']).astype('object')
    q_pays = {
        'Laboratory Technician': 2705.0, 
        'Manufacturing Director': 5824.4000000000015, 
        'Sales Executive': 5675.8, 
        'Research Scientist': 2693.4, 
        'Sales Representative': 2325.8, 
        'Healthcare Representative': 6348.6, 
        'Research Director': 15014.600000000002, 
        'Human Resources': 2741.0, 
        'Manager': 16894.0
    }
    df['AboveQPay'] = df.apply(lambda x: 'No' if x.MonthlyIncome < q_pays.get(x.JobRole) else 'Yes', axis = 1)
    #df['WorkFactors'] = (df.EnvironmentSatisfaction+df.JobSatisfaction+df.JobInvolvement) / 15
    cols_to_drop = [
                    'Id',
                    'Behaviour',
                    'PerformanceRating',
                    'Gender',
                    'Education', 
                    'Department',
                    'EmployeeNumber',
                    'PercentSalaryHike',
                    'YearsInCurrentRole',
                    'YearsSinceLastPromotion',
                    'YearsWithCurrManager',
                    'TrainingTimesLastYear',
                    'EducationField',
                    'StockOptionLevel',
                    'TotalWorkingYears',
                    'YearsAtCompany',
                    'NumCompaniesWorked',
                    'Age',
                    'MonthlyIncome',
                    ]
    df.drop(cols_to_drop, inplace = True, axis = 1)
    print('Columns Dropped : {}'.format(cols_to_drop))
    print('Columns in DataFrame: {}'.format(df.columns.to_list()))
    return df

In [304]:
df_cleaned = extract_feature(df)

Columns Dropped : ['Id', 'Behaviour', 'PerformanceRating', 'Gender', 'Education', 'Department', 'EmployeeNumber', 'PercentSalaryHike', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager', 'TrainingTimesLastYear', 'EducationField', 'StockOptionLevel', 'TotalWorkingYears', 'YearsAtCompany', 'NumCompaniesWorked', 'Age', 'MonthlyIncome']
Columns in DataFrame: ['BusinessTravel', 'DistanceFromHome', 'EnvironmentSatisfaction', 'JobInvolvement', 'JobRole', 'JobSatisfaction', 'MaritalStatus', 'OverTime', 'CommunicationSkill', 'OwnStocks', 'PropWorkLife', 'PropExpComp', 'PropRoleComp', 'AgeBar', 'AboveQPay']


In [305]:
df_cleaned.dtypes

BusinessTravel              object
DistanceFromHome             int64
EnvironmentSatisfaction      int64
JobInvolvement               int64
JobRole                     object
JobSatisfaction              int64
MaritalStatus               object
OverTime                    object
CommunicationSkill           int64
OwnStocks                   object
PropWorkLife               float64
PropExpComp                float64
PropRoleComp               float64
AgeBar                      object
AboveQPay                   object
dtype: object

### Preprocessing and Model Building Init

In [306]:
df_cleaned.shape

(470, 15)

In [307]:
df_X = df_cleaned.copy()

In [308]:
nominal_type = list(df_X.select_dtypes(include='object').columns)
nominal_type

['BusinessTravel',
 'JobRole',
 'MaritalStatus',
 'OverTime',
 'OwnStocks',
 'AgeBar',
 'AboveQPay']

In [309]:
nominal_type_vals = dict()
for ot in nominal_type:
    nominal_type_vals[ot] = list(df_X[ot].unique())
print(nominal_type_vals)

{'BusinessTravel': ['Travel_Rarely', 'Non-Travel', 'Travel_Frequently'], 'JobRole': ['Research Scientist', 'Sales Executive', 'Manufacturing Director', 'Laboratory Technician', 'Healthcare Representative', 'Human Resources', 'Manager', 'Research Director', 'Sales Representative'], 'MaritalStatus': ['Married', 'Divorced', 'Single'], 'OverTime': ['No', 'Yes'], 'OwnStocks': ['Yes', 'No'], 'AgeBar': ['Mid', 'Old', 'Young'], 'AboveQPay': ['No', 'Yes']}


In [310]:
numerical_type = list(df_X.select_dtypes(exclude='object').columns)
numerical_type

['DistanceFromHome',
 'EnvironmentSatisfaction',
 'JobInvolvement',
 'JobSatisfaction',
 'CommunicationSkill',
 'PropWorkLife',
 'PropExpComp',
 'PropRoleComp']

In [311]:
ordinal_type = list()
ordinal_columns_dataset = ['EnvironmentSatisfaction',
                           'JobInvolvement',
                           'JobSatisfaction',
                           'Education',
                           'Behaviour',
                           'CommunicationSkill',
                           'PerformanceRating',
                           'StockOptionLevel',
                          ]
for col in ordinal_columns_dataset:
    if col in numerical_type:
        numerical_type.remove(col)
        ordinal_type.append(col)
        
ordinal_type

['EnvironmentSatisfaction',
 'JobInvolvement',
 'JobSatisfaction',
 'CommunicationSkill']

In [312]:
final_cols = list(df_X.columns)
final_cols

['BusinessTravel',
 'DistanceFromHome',
 'EnvironmentSatisfaction',
 'JobInvolvement',
 'JobRole',
 'JobSatisfaction',
 'MaritalStatus',
 'OverTime',
 'CommunicationSkill',
 'OwnStocks',
 'PropWorkLife',
 'PropExpComp',
 'PropRoleComp',
 'AgeBar',
 'AboveQPay']

In [313]:
numerical_index = list()
nominal_index = list()
ordinal_index = list()

for col in numerical_type:
    numerical_index.append(final_cols.index(col))
for col in nominal_type:
    nominal_index.append(final_cols.index(col))
for col in ordinal_type:
    ordinal_index.append(final_cols.index(col))

In [314]:
print('Numerical Columns')
for i,col in zip(numerical_index, numerical_type):
    print(i, col)
print('='*50)
print('Nominal Columns')
for i,col in zip(nominal_index, nominal_type):
    print(i, col)
print('='*50)
print('Ordinal Columns')
for i,col in zip(ordinal_index, ordinal_type):
    print(i, col)

Numerical Columns
1 DistanceFromHome
10 PropWorkLife
11 PropExpComp
12 PropRoleComp
Nominal Columns
0 BusinessTravel
4 JobRole
6 MaritalStatus
7 OverTime
9 OwnStocks
13 AgeBar
14 AboveQPay
Ordinal Columns
2 EnvironmentSatisfaction
3 JobInvolvement
5 JobSatisfaction
8 CommunicationSkill


In [315]:
df_X.describe()

,DistanceFromHome,EnvironmentSatisfaction,JobInvolvement,JobSatisfaction,CommunicationSkill,PropWorkLife,PropExpComp,PropRoleComp
count,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000
mean,9.293617,2.708511,2.736170,2.693617,3.051064,0.290479,0.271368,0.482867
std,8.084506,1.089850,0.729057,1.138473,1.413289,0.152715,0.283311,0.270832
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,2.000000,2.000000,2.000000,0.181818,0.090909,0.333333
50%,7.000000,3.000000,3.000000,3.000000,3.000000,0.276820,0.166667,0.500000
75%,15.000000,4.000000,3.000000,4.000000,4.000000,0.386913,0.400000,0.666667
max,29.000000,4.000000,4.000000,4.000000,5.000000,0.689655,2.000000,0.882353


In [316]:
from sklearn.preprocessing import OneHotEncoder
encoder_dic = {}
for col in nominal_index:
    enc = OneHotEncoder(categories='auto',drop='first',sparse = False).fit(df_X.iloc[:,[col]])
    encoder_dic[col] = enc

In [317]:
X_test = df_X.copy()

In [318]:
X_test.shape

(470, 15)

In [319]:
for i, enc in encoder_dic.items():
    temp = pd.DataFrame(enc.transform(X_test.iloc[:,[i]]), columns=[ list(df_X.columns)[i]+'_'+colname for colname in list(enc.categories_[0])[1:]]).reset_index()
    X_test = pd.concat( [X_test, temp], axis = 1 )

In [320]:
X_test.drop(nominal_type, inplace=True, axis=1)

In [321]:
final_cols_encoded = X_test.columns.values
final_cols_encoded

array(['DistanceFromHome', 'EnvironmentSatisfaction', 'JobInvolvement',
       'JobSatisfaction', 'CommunicationSkill', 'PropWorkLife',
       'PropExpComp', 'PropRoleComp', 'index',
       'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely',
       'index', 'JobRole_Human Resources',
       'JobRole_Laboratory Technician', 'JobRole_Manager',
       'JobRole_Manufacturing Director', 'JobRole_Research Director',
       'JobRole_Research Scientist', 'JobRole_Sales Executive',
       'JobRole_Sales Representative', 'index', 'MaritalStatus_Married',
       'MaritalStatus_Single', 'index', 'OverTime_Yes', 'index',
       'OwnStocks_Yes', 'index', 'AgeBar_Old', 'AgeBar_Young', 'index',
       'AboveQPay_Yes'], dtype=object)

In [322]:
X_test = X_test.values

In [323]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [324]:
def test_model_perf(model, X):
    predictions_df = pd.DataFrame(model.predict_proba(X),columns=['No_Attrition','Attrition'], index=range(1,471))
    predictions_df.No_Attrition = predictions_df.No_Attrition.apply(lambda x: np.around(x,7))
    predictions_df.Attrition = predictions_df.Attrition.apply(lambda x: np.around(x,7))
    predictions_df.index.name = 'Id'
    final_df = predictions_df[['Attrition']]
    return final_df

In [325]:
!mkdir submissions

mkdir: submissions: File exists


In [326]:
pkl_filename = "models/Final_RandomForestC.pkl"
sub_filename = "submissions/Final_RandomForestC.csv"
model = RandomForestClassifier()
with open(pkl_filename, 'rb') as file:
    model = pickle.load(file)
predictions_df = test_model_perf(model, X_test)
predictions_df.to_csv(sub_filename)

In [327]:
pkl_filename = "models/Final_RandomForestB.pkl"
sub_filename = "submissions/Final_RandomForestB.csv"
model = RandomForestClassifier()
with open(pkl_filename, 'rb') as file:
    model = pickle.load(file)
predictions_df = test_model_perf(model, X_test)
predictions_df.to_csv(sub_filename)

ValueError: Number of features of the model must match the input. Model n_features is 30 and input n_features is 32 

In [57]:
pkl_filename = "models/Final_RandomForestA.pkl"
sub_filename = "submissions/Final_RandomForestA.csv"
model = RandomForestClassifier()
with open(pkl_filename, 'rb') as file:
    model = pickle.load(file)
predictions_df = test_model_perf(model, X_test)
predictions_df.to_csv(sub_filename)

In [ ]:
pkl_filename = "models/RandomForest.pkl"
sub_filename = "submissions/RandomForest.csv"
model = RandomForestClassifier()
with open(pkl_filename, 'rb') as file:
    model = pickle.load(file)
predictions_df = test_model_perf(model, X_test)
predictions_df.to_csv(sub_filename)